In [49]:

import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB

import warnings
warnings.filterwarnings("ignore")                     #Ignoring unnecessory warnings

import numpy as np                                  #for large and multi-dimensional arrays
import pandas as pd                                 #for data manipulation and analysis
import nltk                                         #Natural language processing tool-kit

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer

from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer          #For TF-IDF
from gensim.models import Word2Vec                                   #For Word2Vec

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.layers import SpatialDropout1D


In [62]:
import os
import pandas as pd

df = pd.read_csv("./data/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1')
df = df.dropna(axis=1)
df = df.drop(columns=['rater1_domain1', 'rater2_domain1'])
df = df.drop(columns=['essay_id', 'essay_set'])
df.head()

,essay,domain1_score
0,"Dear local newspaper, I think effects computer...",8
1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,"Dear @LOCATION1, I know having computers has a...",8


In [63]:
list_1 = []
list_2 = []
list_3 = []
list_4 = []
list_5 = []
list_6 = []
list_7 = []
list_8 = []
list_9 = []
list_10 = []

In [64]:
data = []
num = [1,2,3,4,5,6,7,8,9,10]
count = 0
for i,j in zip(range(len(df)),df["essay"]):
    if df["domain1_score"][i] == 1:
        list_1.append(j)
    if df["domain1_score"][i] == 2:
        list_2.append(j)
    if df["domain1_score"][i] == 3:
        list_3.append(j)
    if df["domain1_score"][i] == 4:
        list_4.append(j)
    if df["domain1_score"][i] == 5:
        list_5.append(j)
    if df["domain1_score"][i] == 6:
        list_6.append(j)
    if df["domain1_score"][i] == 7:
        list_7.append(j)
    if df["domain1_score"][i] == 8:
        list_8.append(j)
    if df["domain1_score"][i] == 9:
        list_9.append(j)
    if df["domain1_score"][i] == 10:
        list_10.append(j)
   

In [65]:
import itertools
data = list(itertools.chain(list_1,list_2,list_3,list_4,list_5,list_6,list_7,list_8,list_9,list_10))
len(data)

10323

In [66]:
def gen_num(num,length):
    num_list = [num]*length
    return num_list

In [67]:
score_1 = gen_num(1,len(list_1))
score_2 = gen_num(2,len(list_2))
score_3 = gen_num(3,len(list_3))
score_4 = gen_num(4,len(list_4))
score_5 = gen_num(5,len(list_5))
score_6 = gen_num(6,len(list_6))
score_7 = gen_num(7,len(list_7))
score_8 = gen_num(8,len(list_8))
score_9 = gen_num(9,len(list_9))
score_10 = gen_num(10,len(list_10))

score = list(itertools.chain(score_1,score_2,score_3,score_4,score_5,score_6,score_7,score_8,score_9,score_10))
len(score)


10323

In [68]:
# dictionary of lists 
dictnary = {'essay': data, 'score': score}     
df = pd.DataFrame(dictnary) 
df 

,essay,score
0,Write a persuasive essay to a newspaper reflec...,1
1,If the people that are publishing and writing ...,1
2,I do think that there should be a censorship i...,1
3,I belive that they need to keep every thing an...,1
4,Hi my name is @CAPS1 @CAPS2 @PERSON1 and I'm h...,1
...,...,...
10318,A time I was patient was when I had to be pati...,10
10319,Being patient is hard to be. First of all its ...,10
10320,One day I was watching @CAPS1. I was bored the...,10
10321,My own way about patient is that when somebody...,10


In [69]:
stop_words = set(stopwords.words('english'))
len(stop_words) #finding stop words

179

In [70]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
snow = nltk.stem.SnowballStemmer('english')

corpus = []
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['essay'][i])
    review = review.lower()
    review = review.split()
    
    review = [snow.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

KeyboardInterrupt: 

In [71]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
type(onehot_repr)

list

In [15]:
sent_length=400
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3159 4307 4680]
 [   0    0    0 ... 3966 1000 1000]
 [   0    0    0 ... 2616 3822 1774]
 ...
 [   0    0    0 ... 2937 1774  216]
 [   0    0    0 ... 2535 4850 1903]
 [   0    0    0 ...  131  328  216]]


In [36]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(10,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 40)           200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400, 40)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 257,410
Trainable params: 257,410
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
embedded_docs.shape

(10323, 400)

In [51]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=embedded_docs.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(10,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 400, 40)           200000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 400, 40)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 196)               185808    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1970      
Total params: 387,778
Trainable params: 387,778
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
df_x = df['essay']
df_y = df['score']

Y = pd.get_dummies(df['score']).values


In [43]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(df_y)

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final,Y, test_size=0.2, random_state=42)

In [54]:
#we are feeding the 
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,batch_size=64)

Epoch 1/100
109/130 [========================>.....] - ETA: 1:38 - loss: 1.7578 - accuracy: 0.2952

KeyboardInterrupt: 

# ML begins here

In [72]:
final_tf = df_x
tf_idf = TfidfVectorizer(ngram_range=(1,2),max_features=5000)
tf_data = tf_idf.fit_transform(final_tf)
print(tf_data[1])

  (0, 2809)	0.10767927786849732
  (0, 4040)	0.10995545608632183
  (0, 2895)	0.03942897433369238
  (0, 3987)	0.1068413162788372
  (0, 1520)	0.05534913014730051
  (0, 3295)	0.07675720782532444
  (0, 4185)	0.06825750580876035
  (0, 4152)	0.05364428201361983
  (0, 1920)	0.062122322342736014
  (0, 4215)	0.06849166598269865
  (0, 4050)	0.08064361386468692
  (0, 2332)	0.0704790320532723
  (0, 1147)	0.09225535678240561
  (0, 2949)	0.09679360949925747
  (0, 586)	0.0819980618380052
  (0, 2029)	0.10643605017633438
  (0, 3399)	0.10184047142518715
  (0, 251)	0.09055543336548616
  (0, 3845)	0.1068413162788372
  (0, 2331)	0.07850947458984808
  (0, 2190)	0.08609037417656265
  (0, 4362)	0.054171116232681595
  (0, 4684)	0.06191879022494161
  (0, 717)	0.08770628034420633
  (0, 4274)	0.10199778465002214
  :	:
  (0, 4149)	0.11650229063022867
  (0, 4109)	0.08567968472730202
  (0, 370)	0.10663758193931641
  (0, 4140)	0.18407608500769523
  (0, 4919)	0.07815438753921669
  (0, 315)	0.132406540832952
  (0, 3057)

In [73]:
import pickle

v = pickle.load(open('tfidf_vectorizer.pkl','rb'))

In [28]:
v = pickle.load(open('tfidf_vectorizer.pkl','rb'))

In [29]:
tf_data.get_shape()

(10323, 5000)

In [30]:
features = tf_idf.get_feature_names()
features[:10]

['1937',
 'abandoned',
 'ability',
 'ability to',
 'able',
 'able to',
 'about',
 'about computers',
 'about far',
 'about faraway']

In [94]:
SEED =42
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(tf_data,df_y,test_size = 0.2,random_state=SEED)

In [92]:
y_train.shape

(8258,)

In [32]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(x_train, y_train)

y_pred_nb=nb_model.predict(x_val)

In [33]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred_nb)

0.42469733656174335

In [61]:
y_val.shape

(2065,)

# Random Forest

In [95]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(x_train,y_train)
y_pred_rf = rf_model.predict(x_val)
accuracy_score(y_val,y_pred_rf)

0.5215496368038741

In [104]:
pickle.dump(rf, open("./models/rf-0.5215.pkl", "wb"))

# RF with random search cv

In [99]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]



random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

cls_rf_rscv = RandomForestClassifier()
from sklearn.model_selection import RandomizedSearchCV

random_search_rf = RandomizedSearchCV(cls_rf_rscv, random_grid,n_iter=5, n_jobs=1, cv=2,verbose=2)
random_search_rf.fit(x_train,y_train)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] n_estimators=1400, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=None, bootstrap=False 
[CV]  n_estimators=1400, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=None, bootstrap=False, total= 5.8min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.9min remaining:    0.0s


[CV] n_estimators=1400, min_samples_split=2, min_samples_leaf=2, max_features=sqrt, max_depth=None, bootstrap=False 


Exception ignored in: <function ConcreteFunctionGarbageCollector.__del__ at 0x000001AD3D49DD08>
Traceback (most recent call last):
  File "C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\eager\function.py", line 3790, in __del__
    pass
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
random_search_rf.best_params_

In [100]:
x_train.shape

(8258, 5000)

In [101]:
SEED=42
base_model = RandomForestClassifier(bootstrap= False,
                                    max_depth= 40,
                                    max_features= 'sqrt',
                                    min_samples_leaf= 2,
                                    min_samples_split= 2,
                                    n_estimators= 2000,
                                    random_state = SEED)
base_model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=40, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [102]:
y_pred_rf_rscv = base_model.predict(x_val)

In [103]:
# y_pred_rf_rscv = rf_rscv_model.predict(x_val)
accuracy_score(y_val,y_pred_rf_rscv)

0.6077481840193705

In [ ]:
pickle.dump(base_model, open("./models/rf-0.607.pkl", "wb"))

# do weighted average

In [ ]:
from similarity_score_estimation import get_similarity_score
ans1 = "AI is the understanding that machines can interpret, mine, and learn from external data in a way where said machines functionally imitate cognitive practices normally attributed to humans. Artificial intelligence is based on the notion that human thought processes have the ability to both be replicated and mechanized. The history of artificial intelligence dates back to antiquity with philosophers mulling over the idea that artificial beings, mechanical men, and other automatons had existed or could exist in some fashion. To know more about the history you can also visit Analytics Jobs as I got to know about the history of AI and the changes that are happening daily.Thanks to early thinkers, artificial intelligence became increasingly more tangible throughout the 1700s and beyond. Philosophers contemplated how human thinking could be artificially mechanized and manipulated by intelligent non-human machines. The thought processes that fuelled interest in AI originated when classical philosophers, mathematicians, and logicians considered the manipulation of symbols (mechanically), eventually leading to the invention of a programmable digital computer, the Atanasoff Berry Computer (ABC) in the 1940s. This specific invention inspired scientists to move forward with the idea of creating an “electronic brain,” or an artificially intelligent being.In 1955 a leading computer expert, John McCarthy, wrote a proposal to hold a two-month workshop with top computer scientists. The goal was to enable computers to simulate the basics of human intelligence. They sincerely thought that two months would be plenty of time to address all aspects of the problem and start a new era of human intelligence in a machine.Obviously they didn’t succeed, but the event launched the first period of intense AI research. It led to many highly useful developments, but putting the basics of human intelligence into a computer proved to be far more difficult than anyone imagined. After sixty-four years we’re barely able to reproduce the simplest biological intelligence, let alone anything approaching human level. The top researchers currently say that even a primitive artificial general intelligence is a long way off and it’s unknown how it would ever compare to human intelligence."
ans2 = 'McCarthy invented Artificial Intelligence.The term artificial intelligence was first coined by John McCarthy in 1956 when he held the first academic conference on the subject. But the journey to understand if machines can truly think began much before that. Introduction of AI: Logic was introduced into AI research as early as 1958, by John McCarthy in his Advice Taker proposal. In 1963, J. Alan Robinson had discovered a simple method to implement deduction on computers, the resolution and unification algorithm.He created the term "artificial intelligence" and , was a towering figure in computer science at the Stanford University , most of his professional life. In his career, he developed the programming language LISP, played computer chess via telegraph with opponents in Russia and invented computer time-sharing.In proposing the first conference on Artificial Intelligence, McCarthy wrote, "The study is to proceed on the basis of the conjecture that every aspect of learning or any other feature of intelligence can in principle be so precisely described that a machine can be made to simulate it." The subsequent conference is considered a watershed moment in computer science. In late 20th century, McCarthy invented the computer programming language LISP, the second oldest programming language after FORTRAN. LISP is still used today and is the programming language of choice for artificial intelligence.'
sim_score = get_similarity_score(ans1,ans2)


In [74]:
#TESTING WITH SOME DATE
test = 'last updat st june debarghya das debarghyada com fb co dd deedi fb com dd cornel edu educ cornel univers meng comput scienc dec ithaca ny cornel univers bs comput scienc may ithaca ny colleg engin magna cum laud cum gpa major gpa la martinier boy grad may kolkata india link facebook dd github deedyda linkedin debarghyada youtub deedydash twitter debarghya das quora debarghya das coursework graduat advanc machin learn open sourc softwar engin advanc interact graphic compil practicum cloud comput evolutionari comput defend comput network machin learn undergradu inform retriev oper system artifici intellig practicum function program comput graphic practicum research asst teach asst x unix tool script skill program line java shell python javascript ocaml matlab rail latex line c c css php assembl familiar io android mysql experi facebook softwar engin jan present new york ny coursera kpcb fellow softwar engin intern june sep mountain view ca applic chosen kpcb fellow led ship yoda admin interfac new phoenix platform full stack develop wrote review code js use backbon jade stylus requir scala use play googl softwar engin intern may aug mountain view ca work youtub caption team javascript python plan design develop full stack add edit automat speech recognit caption product creat backbon js like framework caption editor phabric open sourc contributor team leader jan may palo alto ca ithaca ny phabric use daili facebook dropbox quora asana creat meme generat php shell led team mit cornel ic london uhelsinki project research cornel robot learn lab research jan jan ithaca ny work ashesh jain prof ashutosh saxena creat planit tool learn larg scale user prefer feedback plan robot trajectori human environ cornel phonet lab head undergradu research mar may ithaca ny led develop quicktongu first ever breakthrough tongu control game prof sam tilsen aid linguist research award top kpcb engin fellow st microsoft code competit cornel nation jump trade challeng finalist th cs cach race bot tournament nd cs biannual intra class bot tournament nation indian nation mathemat olympiad inmo finalist public jain das saxena planit crowdsourc approach learn plan path larg scale prefer feedback tech report icra press tilsen das b mckee real time articulatori biofeedback electromagnet articulographi linguist vanguard press'
snow = nltk.stem.SnowballStemmer('english')

corpus_test = []
# for i in range(0, len(df)):
review = re.sub('[^a-zA-Z]', ' ', test)
review = review.lower()
review = review.split()
    
review = [snow.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)
corpus_test.append(review)

In [75]:
final_tf_test = corpus_test
# tf_idf_test = TfidfVectorizer(ngram_range=(1,2),max_features=5000)
tf_data_test = v.transform(final_tf_test)
tf_data_test.get_shape()

(1, 5000)

In [87]:
from sklearn.ensemble import RandomForestClassifier
loaded_model = pickle.load(open('./models/rf', 'rb'))
result = loaded_model.predict(tf_data_test)
print(result)

ModuleNotFoundError: No module named 'sklearn.ensemble._forest'

In [82]:
from autogradeEngine import predict_score
preds = predict_score(test,"./models/word2vecmodel.bin","./models/final_lstm.h5")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [83]:
preds

4